SHSAT
======


URLs
- [2017- 2018 Class Size Report City Middle School And High School Core Average Class Size](https://data.cityofnewyork.us/Education/2017-2018-Class-Size-Report-City-Middle-School-And/43qn-d6r8): https://data.cityofnewyork.us/resource/43qn-d6r8.csv?$limit=1000000
- [2010-2011 Class Size - School-level detail](https://data.cityofnewyork.us/Education/2010-2011-Class-Size-School-level-detail/urz7-pzb3)

In [28]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
from nycschools import schools

# read SOCRATA_APP_TOKEN from .envpytho file
load_dotenv()
token = os.getenv("SOCRATA_APP_TOKEN")
# os.environ['NYC_SCHOOLS_DATA_DIR'] = '/home/mxc/Projects/school-data-portal/school-data'
os.getenv("NYC_SCHOOLS_DATA_DIR")


'/home/mxc/Documents/nycschools/school-data'

In [1]:
from nycschools import exams, schools
# exams.load_regents_excel()
reg = exams.load_regents()
schools = schools.load_school_demographics()
reg = reg[reg.regents_exam == 'Common Core Algebra']
reg = reg.merge(schools[["dbn", 'boro']], on='dbn', how='inner')
reg["boro","number_tested"].groupby("boro").sum()


FileNotFoundError: [Errno 2] No such file or directory: '/home/mxc/Documents/nycschools/school-data/nyc-regents.csv'

In [5]:
reg = exams.load_regents()
len(reg.regents_exam.unique())

18

In [3]:
import numpy as np
import pandas as pd

# define function to calculate entropy for a row
def entropy(row):
    # get proportions and handle cases where proportion is 0
    p = row[row != 0] / 100

    # calculate entropy
    return -np.sum(p * np.log2(p))

# create example dataframe
df = pd.DataFrame({
    "asian_pct": [10, 20, 30],
    "white_pct": [30, 50, 70],
    "black_pct": [60, 30, 0]
})

# calculate entropy for each row and add to new column "entropy"
df["entropy"] = df.apply(entropy, axis=1)

print(df)

   asian_pct  white_pct  black_pct   entropy
0         10         30         60  1.295462
1         20         50         30  1.485475
2         30         70          0  0.881291


In [33]:
# load the data_urls from config for hs_admissions
data_urls = schools.config.urls["hs_admissions"].data_urls
# get a dataframe from the 2021 url
df = pd.read_csv(data_urls["2021"])
df.head()

,school_dbn,school_name,program_code,selection_criteria
0,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL (01M448),M35C,2018-19 ELA/Math State Test Scores - 20%
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL (01M448),M35C,2018-19 Final Grades - 40%
2,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL (01M448),M35C,2019-20 Marking Period Grades - 40%
3,01M450,EAST SIDE COMMUNITY SCHOOL (01M450),M58A,Essay/Video Response - 100%
4,01M539,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",M29A,2018-19 ELA/Math State Test Scores - 20%


In [34]:
df[["criteria", "percent"]] = df.selection_criteria.str.split(" - ", expand=True)
df["percent"] = df.percent.str.replace("%", "").astype(float) / 100
df.drop("selection_criteria", axis=1, inplace=True)


In [35]:

list(df.criteria.unique())
# df.head()
vals = {
    '2018-19 ELA/Math State Test Scores': "7th grade ELA/Math Scores",
    '2018-19 Final Grades': "7th grade final grades",
    '2019-20 Marking Period Grades': "8th grade fall grades",
    'Essay/Video Response': "Essay/Video Response",
    'Humanities Response': "Humanities Response",
    'STEM Response': "STEM Response",
    'Video Response': "Video Response",
    'Interview': "Interview",
    'Online Admissions Activity': "Online Admissions Activity",
    'Essay': "Essay",
    'Interview/Online Form': "Interview/Online Form",
    'Artifact 1': "Artifact 1",
    'Artifact 2': "Artifact 2",
    'Entrance Assessment: Math': "Entrance Assessment: Math",
    'Written Exercise': "Written Exercise"
}
df.criteria = df.criteria.map(vals)


,school_dbn,school_name,program_code,criteria,percent
0,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL (01M448),M35C,7th grade ELA/Math Scores,0.2
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL (01M448),M35C,7th grade final grades,0.4
2,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL (01M448),M35C,8th grade fall grades,0.4
3,01M450,EAST SIDE COMMUNITY SCHOOL (01M450),M58A,Essay/Video Response,1.0
4,01M539,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",M29A,7th grade ELA/Math Scores,0.2


In [36]:

df["admission year"] = 2021
df.head()

,school_dbn,school_name,program_code,criteria,percent,admission year
0,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL (01M448),M35C,7th grade ELA/Math Scores,0.2,2021
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL (01M448),M35C,7th grade final grades,0.4,2021
2,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL (01M448),M35C,8th grade fall grades,0.4,2021
3,01M450,EAST SIDE COMMUNITY SCHOOL (01M450),M58A,Essay/Video Response,1.0,2021
4,01M539,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",M29A,7th grade ELA/Math Scores,0.2,2021


In [38]:
data = schools.load_hs_directory(2020)

In [46]:
cols = list(data.columns)
[c for c in cols if "admiss" in c.lower()]
data.admissionspriority12.unique()

array([nan, 'Priority to District 6 students or residents',
       'Priority to Brooklyn students or residents',
       'Open to New York City residents',
       'Priority to Bronx students or residents',
       'Priority to continuing 8th graders',
       'Priority to Staten Island students or residents',
       'Priority to Queens students or residents',
       'Priority to Manhattan students or residents',
       'Priority to students who live in the zoned area',
       'Priority to Manhattan students or residents who have lived in the United States two years or fewer and whose home language is Spanish',
       'Open to New York City residents whose home language is Chinese (Mandarin)',
       'Priority to students who reside in the geographical catchment area',
       'Priority to students living in the geographic zone',
       'Priority to Districts 13, 14, 15 and 16 students or residents',
       'Priority to students who live in the zoned area for up to 50% of the seats',
      

In [16]:
# 2021 DOE High School Directory
url = "https://data.cityofnewyork.us/resource/8b6c-7uty.csv?$limit=10000"
hs21 = pd.read_csv(url)


In [17]:
url = "https://data.cityofnewyork.us/resource/23z9-6uk9.csv?$limit=10000"
hs20 = pd.read_csv(url)

In [26]:
# hs21.columns & hs20.columns
len(hs21.columns.difference(hs20.columns))

116

In [26]:
from nycschools import schools
df = schools.load_school_demographics()
df.columns

Index(['dbn', 'beds', 'district', 'geo_district', 'boro', 'school_name',
       'short_name', 'ay', 'year', 'total_enrollment',
       'grade_3k_pk_half_day_full', 'grade_k', 'grade_1', 'grade_2', 'grade_3',
       'grade_4', 'grade_5', 'grade_6', 'grade_7', 'grade_8', 'grade_9',
       'grade_10', 'grade_11', 'grade_12', 'female_n', 'female_pct', 'male_n',
       'male_pct', 'asian_n', 'asian_pct', 'black_n', 'black_pct',
       'hispanic_n', 'hispanic_pct', 'multi_racial_n', 'multi_racial_pct',
       'native_american_n', 'native_american_pct', 'white_n', 'white_pct',
       'missing_race_ethnicity_data_n', 'missing_race_ethnicity_data_pct',
       'swd_n', 'swd_pct', 'ell_n', 'ell_pct', 'poverty_n', 'poverty_pct',
       'eni_pct', 'clean_name', 'zip'],
      dtype='object')

In [ ]:

# Define the API endpoint URL
url = "https://data.cityofnewyork.us/api/catalog/v1?limit=10"

# Send a GET request to the API endpoint and get the JSON response
headers = {
    "X-App-Token": token
}

response = requests.get(url, headers=headers)
data = response.json()

results = data["results"]
len(results)

In [38]:
datasets = {result["metadata"]["domain"]: result for result in results}
datasets.keys()

dict_keys(['www.dallasopendata.com', 'data.ny.gov', 'data.ct.gov', 'data.cdc.gov', 'opendata.howardcountymd.gov', 'data.cityofchicago.org', 'data.nj.gov', 'data.wa.gov', 'data.cityofnewyork.us', 'data.nashville.gov', 'data.memphistn.gov', 'www.datos.gov.co', 'healthdata.gov', 'health.data.ny.gov', 'data.mo.gov', 'datahub.transportation.gov', 'data.montgomerycountymd.gov', 'data.fortworthtexas.gov', 'data.pa.gov', 'data.kingcounty.gov', 'data.lacity.org', 'data.colorado.gov', 'data.virginia.gov', 'data.calgary.ca', 'data.cambridgema.gov', 'data.providenceri.gov', 'data.sfgov.org', 'insights.cincinnati-oh.gov', 'data.pr.gov', 'data.delaware.gov', 'priv-data.ojp.usdoj.gov', 'datahub.usac.org', 'data.vermont.gov', 'data.bts.gov', 'datacatalog.cookcountyil.gov', 'data.cincinnati-oh.gov', 'data.oregon.gov', 'data.lacounty.gov', 'data.edd.ca.gov', 'data.energystar.gov', 'data.norfolk.gov', 'data.hawaii.gov', 'data.cityofgainesville.org', 'data.hartford.gov', 'internal.chattadata.org', 'intern

In [ ]:

# do a dict comprehension to add each item in results keyed by metadata.domain
datasets = {result["metadata"]["domain"]: result for result in results}


# Extract the intersecting keys from the results dictionary
keys = set(results[0].keys())
for result in results:
    keys &= set(result.keys())

# Create a list of dictionaries with only the intersecting keys
filtered_results = []
for result in results:
    filtered_result = {}
    for key in keys:
        filtered_result[key] = result[key]
    filtered_results.append(filtered_result)

# Create a DataFrame from the list of dictionaries
# df = pd.DataFrame(filtered_results)
# df.head()
datasets.keys()
len(results)

In [ ]:
import pandas as pd

# meta = "https://data.cityofnewyork.us/api/views/metadata/v1"
# meta = "https://data.cityofnewyork.us/api/catalog/v1.csv?ids=*"
# get a list of all of the data sets in cityofnewyork socrata
meta = "https://data.cityofnewyork.us/api/catalog/v1?domains=data.cityofnewyork.us"
df = pd.read_json(meta, lines=True)
meta.head()

In [2]:

{
    "doe_22": "https://infohub.nyced.org/docs/default-source/default-document-library/updated2023_avg_classsize_schl.xlsx",

}

url = "https://data.cityofnewyork.us/resource/sgr7-hhwp.csv?$limit=1000000"

# nyc = pd.read_csv(url)
# print(df.columns)
# df.grade_level.unique()
# df.head()

url = "https://infohub.nyced.org/docs/default-source/default-document-library/updated2023_avg_classsize_schl.xlsx"
xls = pd.read_excel(url, sheet_name=None)
# first sheet is k-8
k8 = xls["K-8 Avg"]
hs = xls["MS HS Avg"]
ptr = xls["PTR"]
k8.head()

/home/mxc/.virtualenvs/nycschools/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/mxc/.virtualenvs/nycschools/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/mxc/.virtualenvs/nycschools/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,DBN,School Name,Grade Level,Program Type,Number of Students,Number of Classes,Average Class Size,Minimum Class Size,Maximum Class Size
0,01M015,PS 015 ROBERTO CLEMENTE,K,G&T,11,1,11.0,<15,<15
1,01M015,PS 015 ROBERTO CLEMENTE,K,ICT,25,1,25.0,25,25
2,01M015,PS 015 ROBERTO CLEMENTE,1,G&T,14,1,14.0,<15,<15
3,01M015,PS 015 ROBERTO CLEMENTE,1,ICT,17,1,17.0,17,17
4,01M015,PS 015 ROBERTO CLEMENTE,2,G&T,7,1,7.0,<15,<15


SHSAT
======

In [17]:
# show the intersecting columns
print(set(k8.columns) - set(hs.columns))
print(set(hs.columns) - set(k8.columns))
print(len(hs.columns) , len(k8.columns))

set()
{'Subject', 'Department'}
11 9


In [23]:
k8["Subject"] = "K8"
k8["Department"] = "K8"
df = pd.concat([k8, hs], axis=0).head()
df["ay"] = 2022
cols = {
    "DBN": "dbn",
    "School Name": "school_name",
    "Grade Level": "grade",
    "Program Type": "program_type",
    "Number of Students": "students_n",
    "Number of Classes": "classes_n",
    "Average Class Size": "avg_class_size",
    "Subject": "subject",
    "Department": "dpt",
    "Minimum Class Size": "min_class_size",
    "Maximum Class Size": "max_class_size"
}
df = df.rename(columns=cols)
# drop school_name
df = df.drop(columns=["school_name"])
df.head()
df.to_csv("/opt/nycschools/nyc_class_size.csv", index=False)

In [5]:
df = pd.read_csv("https://data.cityofnewyork.us/resource/43qn-d6r8.csv?$limit=1000000")
print(df.columns)
df

Index(['grade_level', 'program_type', 'department', 'subject',
       'number_of_students', 'number_of_classes', 'average_class_size'],
      dtype='object')


,grade_level,program_type,department,subject,number_of_students,number_of_classes,average_class_size
0,HS Core,Gen Ed,English,HS English 09,48093,1897,25.4
1,HS Core,Gen Ed,English,HS English 10,46274,1762,26.3
2,HS Core,Gen Ed,English,HS English 11,45892,1780,25.8
3,HS Core,Gen Ed,English,HS English 12,44382,1735,25.6
4,HS Core,Gen Ed,English,HS English Other,75065,3458,21.7
...,...,...,...,...,...,...,...
95,MS Core,SC,Math,MS Math Other,598,60,10.0
96,MS Core,SC,Science,MS Science Core,10747,1095,9.8
97,MS Core,SC,Science,MS Science Other,210,23,9.1
98,MS Core,SC,Social Studies,MS Social Studies Core,10663,1079,9.9


In [2]:
from nycschools import config, geo, dataloader, schools


a = schools.load_school_demographics()
b = geo.get_locations()
c = b.merge(a, on="dbn", how="inner")
# c.explore()
# b.open_date.astype("datetime64[ns]").dt.year
b.fiscal_year


0       2020
1       2020
2       2020
3       2020
4       2020
        ... 
2185    2020
2186    2020
2187    2020
2188    2020
2189    2020
Name: fiscal_year, Length: 2190, dtype: int64